In [1]:
import sqlalchemy as db
import pandas as pd
params = {
    'host': '34.217.89.93',  # Replace with your own database IP Address.
    'user': 'ubuntu',
    'port': 5432
}
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/baseball'
engine = db.create_engine(connection_string)
connection = engine.connect()

In [2]:
metadata = db.MetaData()
fielding = db.Table('fielding', metadata, autoload=True, autoload_with=engine)
query = db.select([fielding.c.playerID, \
                  db.func.min(fielding.c.yearID).label('FirstYearPlayed'), \
                  db.func.max(fielding.c.yearID).label('LastYearPlayed')]).\
        group_by(fielding.c.playerID)
results = connection.execute(query)

In [3]:
df = pd.DataFrame(results)
df.columns = results.keys()
print(df.shape)
df.head()

(17981, 3)


,playerID,FirstYearPlayed,LastYearPlayed
0,jacobar01,1939,1939
1,kriegbi01,1884,1887
2,peppela01,1954,1957
3,loesbi01,1950,1961
4,cathete01,1912,1915


In [4]:
print(query)

SELECT fielding."playerID", min(fielding."yearID") AS "FirstYearPlayed", max(fielding."yearID") AS "LastYearPlayed" 
FROM fielding GROUP BY fielding."playerID"


In [5]:
results = connection.execute('SELECT fielding."playerID", \
                             min(fielding."yearID") AS "FirstYearPlayed", \
                             max(fielding."yearID") AS "LastYearPlayed" \
                             FROM fielding GROUP BY fielding."playerID"')

In [6]:
df = pd.DataFrame(results)
df.columns = results.keys()
print(df.shape)
df.head()

(17981, 3)


,playerID,FirstYearPlayed,LastYearPlayed
0,jacobar01,1939,1939
1,kriegbi01,1884,1887
2,peppela01,1954,1957
3,loesbi01,1950,1961
4,cathete01,1912,1915


In [7]:
from sqlalchemy.orm import sessionmaker
# create a configured "Session" class
Session = sessionmaker(bind=engine)
# create a Session
session = Session()       
playRange = session.query(fielding.c.playerID.label('playerID'), \
                          db.func.min(fielding.c.yearID).label('FirstYearPlayed'), \
                          db.func.max(fielding.c.yearID).label('LastYearPlayed')).\
            group_by(fielding.c.playerID).subquery()
query = db.select([playRange])
results = connection.execute(query).fetchall()

In [8]:
df = pd.DataFrame(results)
df.columns = results[0].keys()
print(df.shape)
df.head()

(17981, 3)


,playerID,FirstYearPlayed,LastYearPlayed
0,jacobar01,1939,1939
1,kriegbi01,1884,1887
2,peppela01,1954,1957
3,loesbi01,1950,1961
4,cathete01,1912,1915


Suggestion from Roberto:

In [12]:
pd.read_sql_query(query, connection)

,playerID,FirstYearPlayed,LastYearPlayed
0,jacobar01,1939,1939
1,kriegbi01,1884,1887
2,peppela01,1954,1957
3,loesbi01,1950,1961
4,cathete01,1912,1915
...,...,...,...
17976,halldr01,1986,1990
17977,peppedo01,1966,1966
17978,sembeca01,1965,1970
17979,zinnfr01,1888,1888


In [13]:
print(query)

SELECT anon_1."playerID", anon_1."FirstYearPlayed", anon_1."LastYearPlayed" 
FROM (SELECT fielding."playerID" AS "playerID", min(fielding."yearID") AS "FirstYearPlayed", max(fielding."yearID") AS "LastYearPlayed" 
FROM fielding GROUP BY fielding."playerID") AS anon_1


In [23]:
query = 'SELECT fielding."playerID" AS "playerID",\
                min(fielding."yearID") as "FirstYearPlayed",\
                max(fielding."yearID") as "LastYearPlayed" \
                FROM fielding GROUP BY fielding."playerID"'
pd.read_sql_query(query, connection)

,playerID,FirstYearPlayed,LastYearPlayed
0,jacobar01,1939,1939
1,kriegbi01,1884,1887
2,peppela01,1954,1957
3,loesbi01,1950,1961
4,cathete01,1912,1915
...,...,...,...
17976,halldr01,1986,1990
17977,peppedo01,1966,1966
17978,sembeca01,1965,1970
17979,zinnfr01,1888,1888
